In [16]:
from eventregistry import *
from newspaper import Article
from threading import Thread, Lock
from watson_developer_cloud import NaturalLanguageUnderstandingV1
import watson_developer_cloud.natural_language_understanding.features.v1 as Features
import numpy as np
import pandas as pd
import random
import json


# Print a list of recently added articles mentioning entered words
api_key = 'eda39267-9017-481a-860d-0b565c6d8bf3'
er = EventRegistry(apiKey = api_key)

global_df = pd.DataFrame()
mutex = Lock()

def get_articles(keywords,count=199): 
    global global_df
    q = QueryArticlesIter(keywords=QueryItems.AND(keywords))
    q.setRequestedResult(RequestArticlesInfo(count= count, sortBy="sourceImportance"))
    print keywords
    
    x = 0
    
    local_df = pd.DataFrame()
    
    res = er.execQuery(q)
    for article in res['articles']['results']:
        data = {
            'source': article['source']['title'].encode('utf-8'),
#             'title' : article['title'].encode('utf-8'),
            'url' : article['url'].encode('utf-8'),
            'text' : article['body'].encode('utf-8')
        }
        local_df = pd.concat([local_df, pd.DataFrame(data,index=[x])])
        x += 1
        
    mutex.acquire()
    try:
        global_df = pd.concat([global_df,local_df])
    finally:
        mutex.release()

def get_search_params(keywords):
    search_params = []
    while len(keywords) != 0:
        # Randomly select 3 words
        rm = random.sample(keywords,3)
        # add the list of 3 words to the searchable list
        search_params.append(rm)
        # remove words from the list
        for word in rm:
            keywords.remove(word)

        # put 1 or 2 random words back
        # if 3 words left just append to search_params
        if len(keywords) is 3:
            search_params.append(keywords)
            keywords = []
        # if no words left just exit
        elif len(keywords) is 0:
            keywords = []
        # if 1 word left, append 2 and search_params
        elif len(keywords) is 1:
            keywords.append(random.sample(rm,2)[0:2])
        else:
            keywords.append(random.sample(rm,1)[0])
    return search_params

def get_keywords(user_url):
    url = user_url.decode('utf-8')
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    keywords = article.keywords
    kl = []
    for word in keywords:
        kl.append(word.encode('utf-8'))
    return kl

def watson(user_url):
    natural_language_understanding = NaturalLanguageUnderstandingV1(
      username="09b56387-57ee-4390-9365-a07a37706fb4",
      password="ISoTe5EueZJp",
      version="2017-02-27")

    response = natural_language_understanding.analyze(
      url=user_url,
      features=[
        Features.Keywords(
          emotion=False,
          sentiment=False,
            limit=15
        )
      ]
    )
    keywords = []
    for keyword in response['keywords']:
        if keyword['relevance'] > 0.80 and len(keywords) < 8:
            keywords.append(keyword['text'].encode('utf-8'))
    return keywords

class myThread(threading.Thread):
    def __init__(self, query):
        threading.Thread.__init__(self)
        self.query = query

    def run(self):
        get_articles(self.query)
        
def web_scrape(url):
    global global_df
    kl = get_keywords(url)
    params = get_search_params(kl)

    index = 0
    threads = []
    
    for query in params:
        threads.append(myThread(query))
        threads[index].start()
        index += 1
    for thread in threads:
        thread.join()

    global_df = global_df.reset_index(drop=True)
#     df.to_json(orient='index')
    global_df.to_csv('articles.csv')
    print global_df
    
def watson_scrape(url):
    global global_df
    keywords = watson(url)
    
    index = 0
    threads = []
    
    for query in keywords:
        threads.append(myThread(query))
        threads[index].start()
        index += 1
    for thread in threads:
        thread.join()
    global_df = global_df.reset_index(drop=True)
    global_df.to_csv('watson_articles.csv')
    print global_df
    
def watson_claim(claim):
    natural_language_understanding = NaturalLanguageUnderstandingV1(
      username="09b56387-57ee-4390-9365-a07a37706fb4",
      password="ISoTe5EueZJp",
      version="2017-02-27")

    response = natural_language_understanding.analyze(
      url=user_url,
      features=[
        Features.Keywords(
          emotion=False,
          sentiment=False,
            limit=15
        )
      ]
    )
    keywords = []
    for keyword in response['keywords']:
        if keyword['relevance'] > 0.80 and len(keywords) < 8:
            keywords.append(keyword['text'].encode('utf-8'))
    return keywords

def watson_scrape_claim(claim):
    global global_df
    
    get_articles(claim[:,3])
    keywords = watson_claim(claim)
    
    index = 0
    threads = []
    
    for query in keywords:
        threads.append(myThread(query))
        threads[index].start()
        index += 1
    for thread in threads:
        thread.join()
    global_df = global_df.reset_index(drop=True)
    global_df.to_csv('watson_articles.csv')
    print global_df
    
def main():
    url = 'http://abcnews.go.com/US/wireStory/hurricanes-teach-us-ap-finds-fast-coastal-growth-49893843'
#     web_scrape(url)
    watson_scrape(url)

main()

using user provided API key for making requests
Event Registry host: http://eventregistry.org
Natural Hazards Research
coastline counties
South Carolina
latest punishing hurricanes
South Carolina coastline
intense U.S. hurricanes
coastal South Carolina
fastest-growing coastline counties
                            source  \
0               The New York Times   
1                      Mail Online   
2                         ABC News   
3                         NOLA.com   
4                The Seattle Times   
5               San Francisco Gate   
6                       Boston.com   
7                             mySA   
8                  The Denver Post   
9                     Star Tribune   
10                            WFTV   
11            The Kansas City Star   
12                        WRAL.com   
13                    Daily Herald   
14                    Miami Herald   
15                    Fox Business   
16                            WSOC   
17                         W

In [21]:
claim = 

ValueError: Arrays were different lengths: 1262 vs 1

In [38]:
print arts

                  source                                               text  \
0                   test                                            testing   
0     The New York Times  FAIR BLUFF, N.C. -- The flooding is long gone,...   
1     The New York Times  WASHINGTON -- As Hurricane Irma rampaged throu...   
2     The New York Times  CLEWISTON, Fla. -- Vicious hurricanes all in a...   
3     The New York Times  The White House says Trump spoke to the offici...   
4     The New York Times  But even after the storm illustrated the city'...   
5     The New York Times  A hurricane watch was issued for Antigua and B...   
6     The New York Times  About 4 a.m. local time, the storm was 40 mile...   
7     The New York Times  "A lot of people died," Mr. Marte said of Harv...   
8     The New York Times  MEXICO CITY -- Sorry, we have too many trouble...   
9     The New York Times  If you're flying, call your airline to get a s...   
10    The New York Times  Over those years, many thi

In [33]:
arts['articles']['results'][0]['source']['title']
len(arts['articles']['results'])

199

In [24]:
df1 = pd.read_csv('100_articles.csv')
df1.columns = ['id','source','text',"""'title""" 'url']

bodies = df1.loc[:,['id','text']]
bodies.to_csv('bodies.csv')

claim = [claim] * len(global_df.index)
claims = pd.DataFrame(claim)
claims['id'] = range(len(global_df.index))
claims.to_csv('claims.csv')

# we want claim here, not title
# headlines = df1.loc[:,['id','title']]
# headlines.to_csv('headlines.csv')

urls = df1.loc[:,['id','source','url']]
urls.to_csv('url.csv')

In [27]:
claim = "Brazilian junk food"
claim = [claim] * len(df1.index)
claims = pd.DataFrame(claim)
claims['id'] = range(len(df1.index))
claims.to_csv('claims.csv')
# claim = np.array(claim)
# # print type(claim)
# claims = df1.loc[:,['id']]
# claims = pd.concat([claims,pd.DataFrame(claim)])
# claims = pd.DataFrame(claim)
# claims.columns = ['claim']
# claims.to_csv('claims.csv')

,0,id
0,Brazilian junk food,0
1,Brazilian junk food,1
2,Brazilian junk food,2
3,Brazilian junk food,3
4,Brazilian junk food,4
5,Brazilian junk food,5
6,Brazilian junk food,6
7,Brazilian junk food,7
8,Brazilian junk food,8
9,Brazilian junk food,9


In [9]:
import pandas as pd

claims2 = pd.read_csv('claims.csv')
# claims2 = claims2.loc[:,'claim']
claims2.columns = ['id','claim']
# claims2.to_csv('claims.csv')
print claims2

    id                        claim
0    0  US coastal growth continues
1    1  US coastal growth continues
2    2  US coastal growth continues
3    3  US coastal growth continues
4    4  US coastal growth continues
5    5  US coastal growth continues
6    6  US coastal growth continues
7    7  US coastal growth continues
8    8  US coastal growth continues
9    9  US coastal growth continues
10  10  US coastal growth continues
11  11  US coastal growth continues
12  12  US coastal growth continues
13  13  US coastal growth continues
14  14  US coastal growth continues
15  15  US coastal growth continues
16  16  US coastal growth continues
17  17  US coastal growth continues
18  18  US coastal growth continues
19  19  US coastal growth continues
20  20  US coastal growth continues
21  21  US coastal growth continues
22  22  US coastal growth continues
23  23  US coastal growth continues
24  24  US coastal growth continues
25  25  US coastal growth continues
26  26  US coastal growth co

In [15]:
df4 = pd.read_csv('bodies.csv')
df4

,Unnamed: 0,id,text
0,0,0,At least 25 people in one Florida county have ...
1,1,1,"Mr. Northam said in an interview that ""an awak..."
2,2,2,Florida began the colossal task of cleaning up...
3,3,3,The northern eyewall of Hurricane Irma has str...
4,4,4,Rising sea levels and fierce storms have faile...
5,5,5,The largest evacuation in US history is underw...
6,6,6,MIAMI (AP) - The Latest on Hurricane Irma (all...
7,7,7,Excerpts from recent editorials in the United ...
8,8,8,Miami was thrown a lifeline on Saturday as Hur...
9,9,9,WASHINGTON (AP) - The Environmental Protection...


In [71]:
from threading import Thread, Lock

mutex = Lock()

aggregate_keywords = []

class myThread(threading.Thread):
    def __init__(self, url):
        threading.Thread.__init__(self)
        self.url = url

    def run(self):
        get_keywords(self.url)
        
def get_keywords(user_url):
    url = user_url.decode('utf-8')
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()

    keywords = article.keywords
    kl = []
    for word in keywords:
        kl.append(word.encode('utf-8'))
    mutex.acquire()
    try:
        aggregate_keywords.append(kl)
        print 'try'
    finally:
        mutex.release()
        print 'finally'

urls = ['https://www.nytimes.com/2017/09/16/world/asia/kabul-green-zone-afghanistan.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news',
        'http://abcnews.go.com/US/wireStory/irma-shuttered-schools-add-stress-families-49894226?cid=clicksource_4380645_1_hero_headlines_headlines_hed',
       'http://www.cnn.com/2017/09/16/us/chicago-freezer-death/index.html',
       'http://www.cnn.com/2017/09/16/politics/russia-fires-on-us-backed-forces/index.html']

index = 0
threads = []
for url in urls: 
    threads.append(myThread(url))
    threads[index].start()
    index += 1
    print index

for thread in threads:
    thread.join()
print aggregate_keywords

        

1
2
3
4
try
finally
try
finally
try
finally
try
finally[['afghanistan', 'trump', 'zone', 'kabul', 'american', 'embassy', 'green', 'digging', 'mr', 'security', 'afghan', 'expands', 'decade'], ['deir', 'city', 'group', 'isis', 'syrian', 'coalition', 'strike', 'usbacked', 'rebel', 'forces', 'ezzor', 'russian', 'syria', 'sdf'], ['shown', 'death', 'chicago', 'footage', 'hotel', 'freezer', 'surveillance', 'jenkins', 'video', 'martin', 'cnn', 'seen', 'told'], ['selfsustaining', 'school', 'returning', 'keys', 'west', 'florida', 'county', 'officials', 'key', 'water', 'residents', 'south']]



In [3]:
import numpy as np
import pandas as pd
import io, json
dataframe = pd.read_csv('articles.csv')

dataframe = dataframe.loc[:100,['source','text','url']]
dataframe.to_csv('100_articles.csv')